In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torchvision
import cv2

In [53]:
dir = r'C:\works\dacon\img_clf\data\train'

In [54]:
folder_ls = os.listdir(dir)
folder_ls = sorted(folder_ls)
print(len(folder_ls))

396


In [55]:
folder_ls_fl = [folder for folder in folder_ls if not folder.startswith('.')]

In [56]:
len(folder_ls_fl)

396

In [57]:
# make file list each folder

file_ls = []
for folder in folder_ls_fl:
    folder_path = os.path.join(dir, folder)
    file_ls += [os.path.join(folder_path, file) for file in os.listdir(folder_path)]

In [58]:
print(file_ls[:5])

['C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0000.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0001.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0002.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0003.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0004.jpg']


In [59]:
file_ls = sorted(file_ls)

In [60]:
print(file_ls[:5])

['C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0000.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0001.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0002.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0003.jpg', 'C:\\works\\dacon\\img_clf\\data\\train\\1시리즈_F20_2013_2015\\1시리즈_F20_2013_2015_0004.jpg']


In [61]:
# make dataframe

df = pd.DataFrame(file_ls, columns=['image_path'])
df['image_name'] = df['image_path'].apply(lambda x: os.path.basename(x))
df['folder_name'] = df['image_path'].apply(lambda x: os.path.basename(os.path.dirname(x)))

In [62]:
# make label encoding by folder name

df['label'] = df['folder_name'].astype('category').cat.codes

In [63]:
os.makedirs('../data/train_csv', exist_ok=True)

In [64]:
df.to_csv('../data/train_csv/train_file_list.csv', index=False)

In [45]:
df[:100].to_csv('../data/train_csv/train_file_list_100.csv', index=False)

In [1]:
import pandas as pd

In [12]:
train_df = pd.read_csv(r'C:\works\dacon\img_clf\data\train_csv\train_file_list.csv')
test_df = pd.read_csv(r'C:\works\dacon\img_clf\result.csv')

In [13]:
label_dict = train_df[['folder_name', 'label']].drop_duplicates().set_index('label').to_dict()['folder_name']

In [29]:
folder_name = train_df['folder_name'].unique()

In [30]:
folder_name

array(['1시리즈_F20_2013_2015', '1시리즈_F20_2016_2019', '1시리즈_F40_2020_2024',
       '2008_2015_2017', '2시리즈_그란쿠페_F44_2020_2024',
       '2시리즈_액티브_투어러_F45_2019_2021', '2시리즈_액티브_투어러_U06_2022_2024',
       '3008_2세대_2018_2023', '3시리즈_E90_2005_2012', '3시리즈_F30_2013_2018',
       '3시리즈_G20_2019_2022', '3시리즈_G20_2023_2025',
       '3시리즈_GT_F34_2014_2021', '4시리즈_F32_2014_2020',
       '4시리즈_G22_2021_2023', '4시리즈_G22_2024_2025', '5008_2세대_2018_2019',
       '5008_2세대_2021_2024', '5시리즈_F10_2010_2016', '5시리즈_G30_2017_2023',
       '5시리즈_G60_2024_2025', '5시리즈_GT_F07_2010_2017',
       '6시리즈_F12_2011_2018', '6시리즈_GT_G32_2018_2020',
       '6시리즈_GT_G32_2021_2024', '718_박스터_2017_2024', '718_카이맨_2017_2024',
       '7시리즈_F01_2009_2015', '7시리즈_G11_2016_2018', '7시리즈_G11_2019_2022',
       '7시리즈_G70_2023_2025', '8시리즈_G15_2020_2024', '911_2003_2019',
       '911_992_2020_2024', 'A4_B9_2016_2019', 'A4_B9_2020_2024',
       'A5_F5_2019_2024', 'A6_C8_2019_2025', 'A7_2012_2016',
       'A7_4K_2020_2024', 'A8_D5_2

In [14]:
label_dict

{0: '1시리즈_F20_2013_2015',
 1: '1시리즈_F20_2016_2019',
 2: '1시리즈_F40_2020_2024',
 3: '2008_2015_2017',
 4: '2시리즈_그란쿠페_F44_2020_2024',
 5: '2시리즈_액티브_투어러_F45_2019_2021',
 6: '2시리즈_액티브_투어러_U06_2022_2024',
 7: '3008_2세대_2018_2023',
 8: '3시리즈_E90_2005_2012',
 9: '3시리즈_F30_2013_2018',
 10: '3시리즈_G20_2019_2022',
 11: '3시리즈_G20_2023_2025',
 12: '3시리즈_GT_F34_2014_2021',
 13: '4시리즈_F32_2014_2020',
 14: '4시리즈_G22_2021_2023',
 15: '4시리즈_G22_2024_2025',
 16: '5008_2세대_2018_2019',
 17: '5008_2세대_2021_2024',
 18: '5시리즈_F10_2010_2016',
 19: '5시리즈_G30_2017_2023',
 20: '5시리즈_G60_2024_2025',
 21: '5시리즈_GT_F07_2010_2017',
 22: '6시리즈_F12_2011_2018',
 23: '6시리즈_GT_G32_2018_2020',
 24: '6시리즈_GT_G32_2021_2024',
 25: '718_박스터_2017_2024',
 26: '718_카이맨_2017_2024',
 27: '7시리즈_F01_2009_2015',
 28: '7시리즈_G11_2016_2018',
 29: '7시리즈_G11_2019_2022',
 30: '7시리즈_G70_2023_2025',
 31: '8시리즈_G15_2020_2024',
 32: '911_2003_2019',
 33: '911_992_2020_2024',
 34: 'A4_B9_2016_2019',
 35: 'A4_B9_2020_2024',
 36: 'A5_F5_2019_2024',

In [21]:
test_df['label'] = test_df['prediction'].map(lambda x: label_dict.get(x, 'unknown'))

In [22]:
test_df

,image_path,prediction,label
0,TEST_00000.jpg,253,디_올뉴니로_2022_2025
1,TEST_00001.jpg,353,체로키_KL_2019_2023
2,TEST_00002.jpg,146,X5_G05_2024_2025
3,TEST_00003.jpg,185,뉴_A6_2015_2018
4,TEST_00004.jpg,22,6시리즈_F12_2011_2018
...,...,...,...
8253,TEST_08253.jpg,377,트래버스_2020_2023
8254,TEST_08254.jpg,190,뉴_GV80_2024_2025
8255,TEST_08255.jpg,84,GLE_클래스_W167_2019_2024
8256,TEST_08256.jpg,246,더_뉴스포티지R_2014_2016


In [38]:
sub = pd.read_csv(r'C:\works\dacon\img_clf\data\sample_submission.csv')

In [39]:
classes = sorted(train_df['folder_name'].unique())

In [40]:
# sub 초기화
result_df = pd.DataFrame({'ID':sub['ID']})
for cls in classes:
    result_df[cls] = 0

C:\Users\museh\AppData\Local\Temp\ipykernel_14540\2494786763.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[cls] = 0
C:\Users\museh\AppData\Local\Temp\ipykernel_14540\2494786763.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result_df[cls] = 0
C:\Users\museh\AppData\Local\Temp\ipykernel_14540\2494786763.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

In [41]:
result_df

,ID,1시리즈_F20_2013_2015,1시리즈_F20_2016_2019,1시리즈_F40_2020_2024,2008_2015_2017,2시리즈_그란쿠페_F44_2020_2024,2시리즈_액티브_투어러_F45_2019_2021,2시리즈_액티브_투어러_U06_2022_2024,3008_2세대_2018_2023,3시리즈_E90_2005_2012,...,티볼리_에어_2021_2022,파나메라_2010_2016,파나메라_971_2017_2023,파사트_GT_B8_2018_2022,파일럿_3세대_2016_2018,팰리세이드_2019_2022,팰리세이드_LX3_2025,프리우스_4세대_2016_2018,프리우스_4세대_2019_2022,프리우스_C_2018_2020
0,TEST_00000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TEST_00001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TEST_00002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TEST_00003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TEST_00004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8253,TEST_08253,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8254,TEST_08254,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8255,TEST_08255,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8256,TEST_08256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
for idx, row in test_df.iterrows():
    label = row['label']
    if label in result_df.columns:
        result_df.at[idx, label] = 1

In [43]:
result_df = result_df["ID"]

,ID,1시리즈_F20_2013_2015,1시리즈_F20_2016_2019,1시리즈_F40_2020_2024,2008_2015_2017,2시리즈_그란쿠페_F44_2020_2024,2시리즈_액티브_투어러_F45_2019_2021,2시리즈_액티브_투어러_U06_2022_2024,3008_2세대_2018_2023,3시리즈_E90_2005_2012,...,티볼리_에어_2021_2022,파나메라_2010_2016,파나메라_971_2017_2023,파사트_GT_B8_2018_2022,파일럿_3세대_2016_2018,팰리세이드_2019_2022,팰리세이드_LX3_2025,프리우스_4세대_2016_2018,프리우스_4세대_2019_2022,프리우스_C_2018_2020
0,TEST_00000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TEST_00001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TEST_00002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TEST_00003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TEST_00004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8253,TEST_08253,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8254,TEST_08254,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8255,TEST_08255,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8256,TEST_08256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
result_df.to_csv(r'C:\works\dacon\img_clf\data\submission.csv', index=False)

: 